In [1]:
import os
import requests
import pandas as pd
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
from vnstock3 import Vnstock
from vnstock3.explorer.misc.gold_price import *
from vnstock3.explorer.misc.exchange_rate import *
from vnstock3.botbuilder.noti import Messenger
from datetime import datetime,timedelta,date
import matplotlib.pyplot as plt
import json

In [2]:
btmc_goldprice()

ConnectionError: HTTPConnectionPool(host='api.btmc.vn', port=80): Max retries exceeded with url: /api/BTMCAPI/getpricebtmc?key=3kd8ub1llcg9t45hnoh8hmn7t5kc2v (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7342ccdd67a0>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))

In [3]:
date = datetime.today().strftime('%Y-%m-%d')
df = vcb_exchange_rate(date=date)
df['buy _cash'] = df['buy _cash'].replace('-', '0').str.replace(',', '').astype(float)
df = df[df['buy _cash'] != 0]
df['sell'] = df['sell'].str.replace(',', '').astype(float)
df['exchange_rate'] = (df['buy _cash'] + df['sell'])/2
df

,currency_code,currency_name,buy _cash,buy _transfer,sell,date,exchange_rate
2,AUD,AUSTRALIAN DOLLAR,16140.75,"16,303.79",16826.80,2024-11-22,16483.775
3,CAD,CANADIAN DOLLAR,17740.97,"17,920.17",18495.03,2024-11-22,18118.000
4,CHF,SWISS FRANC,27960.44,"28,242.87",29148.88,2024-11-22,28554.660
5,CNY,CHINESE YUAN,3424.74,"3,459.33",3570.30,2024-11-22,3497.520
7,EUR,EURO,25953.07,"26,215.22",27376.05,2024-11-22,26664.560
8,GBP,UK POUND STERLING,31191.25,"31,506.32",32517.01,2024-11-22,31854.130
9,HKD,HONG KONG DOLLAR,3184.60,"3,216.77",3319.96,2024-11-22,3252.280
11,JPY,JAPANESE YEN,158.97,160.58,168.22,2024-11-22,163.595
12,KRW,KOREAN WON,15.69,17.44,18.92,2024-11-22,17.305
19,SGD,SINGAPORE DOLLAR,18411.81,"18,597.79",19194.39,2024-11-22,18803.100


In [4]:
def MFI(stock, start_date, end_date):
    stock = Vnstock().stock(symbol = stock, source = 'VCI')
    df = stock.quote.history(
                            start = start_date, 
                            end = end_date,
                            interval= '1D' 
                            )
    df['Typical Price'] = (df['high'] + df['low'] + df['close'])/3
    df['Money Flow Type'] = ''
    for i in range(0, len(df)-1):
        j = i + 1
        if df.iloc[j, df.columns.get_loc('Typical Price')] >= df.iloc[i, df.columns.get_loc('Typical Price')]:
            df.iloc[j, df.columns.get_loc('Money Flow Type')] = 'P'
        if df.iloc[j, df.columns.get_loc('Typical Price')] < df.iloc[i, df.columns.get_loc('Typical Price')]:
            df.iloc[j, df.columns.get_loc('Money Flow Type')] = 'N'
    df = df[1:]
    df.reset_index(inplace=True)
    negav = df.loc[df['Money Flow Type'] == 'N', 'Typical Price'].sum()
    Posi = df.loc[df['Money Flow Type'] == 'P', 'Typical Price'].sum()
    money_ratio = Posi/negav
    MFI = 100 - 100/(1+money_ratio)
    return {'mfi': MFI, 'data' : df}

In [ ]:
with open('/home/kiendt/code/ETL/KFinace/Key_info.json') as f:
    data = json.load(f)
stock = "MML"
path = f'/home/kiendt/code/ETL/KFinace/data/{stock}.jpg'
start = (datetime.today() - timedelta(days=21)).strftime('%Y-%m-%d')
end = datetime.today().strftime('%Y-%m-%d')
df = MFI(stock,start,end)['data']
base_list = df['time']
for i in base_list:
    start = (i - timedelta(days=21)).strftime('%Y-%m-%d')
    end = i.strftime('%Y-%m-%d')
    MFI_value = MFI(stock,start,end)['mfi']
    df.loc[df['time'] == i,'MFI'] = MFI_value
chart = df.plot(kind = 'line',y = 'MFI', color = 'blue')
for y in [20, 50, 80]:
    chart.axhline(y=y, color='r', linestyle='--', linewidth=1)
for y in [70, 33]:
    chart.axhline(y=y, color='green', linestyle='--', linewidth=1)
plt.savefig(path)
noti = Messenger(platform='telegram', channel=data['telegram']['channel'], token_key=data['telegram']['token'])
noti.send_message(message=f'{stock} information in {end}', file_path= path,title=f'MFI index in {end}')

ConnectionError: HTTPSConnectionPool(host='api.vietcap.com.vn', port=443): Max retries exceeded with url: /data-mt/graphql (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7342cc0bc0a0>: Failed to establish a new connection: [Errno -2] Name or service not known'))